# Homework 2 - Predicting Samsung Cell Phone Prices
BEN BELHASSEN Mohamed Ali, CIAMPANA Lorenzo, FILESI Gianluca, NUBE Giacomo

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm 
from scipy import stats

from sklearn.linear_model import LinearRegression ## Model that will be used in the CV approach
from sklearn.model_selection import cross_val_score ## Different CVs and their loss functions
from sklearn.model_selection import KFold ## To set the k-fold parameters
from sklearn.model_selection import LeaveOneOut ## To set the Loo parameters

from sklearn.preprocessing import PolynomialFeatures

In [2]:
User = 'G' # G or L

if User == 'G':
    path = '/Users/gianlucafilesi/Library/CloudStorage/OneDrive-EDHEC/01 DATA MINING/Data Mining/HW2/data_Samsung_HW2.xlsx'
else:
    path = '/Users/lorenzociampana/Desktop/EDHEC/Data Mining/Sleeping_record_all_zip/'

data = pd.read_excel(path)
data.head()

,phone_name,brand,announcement_date,inches,battery,battery_type,ram(GB),weight(g),storage(GB),price(USD),Resolution_product,days_from_release_date,up_to_8K
0,Galaxy Xcover 4,Samsung,2017-03-01,5.0,2800,Li-Ion,2,172,16,350.0,13.733867,2497,1
1,Galaxy J7 V,Samsung,2017-03-01,5.5,3300,Li-Ion,2,167,16,170.0,13.733867,2497,1
2,Galaxy S8+,Samsung,2017-03-29,6.2,3500,Li-Ion,6,173,64,350.0,15.265343,2469,2
3,Z4,Samsung,2017-05-01,4.5,2050,Li-Ion,1,143,8,240.0,12.858398,2436,1
4,Galaxy J7 Max,Samsung,2017-06-01,5.7,3300,Li-Ion,4,179,32,200.0,14.544797,2405,1


In [3]:
data['battery_type'] = data['battery_type'].apply(lambda x: 1 if x == 'Li-Ion' else 0)
X1 = data[['battery_type','inches','battery','ram(GB)','weight(g)','storage(GB)','Resolution_product','days_from_release_date','up_to_8K']]
X1 = sm.add_constant(X1)
Y = data['price(USD)']

## Model 1

In [4]:
model1 = sm.OLS(Y,X1).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             price(USD)   R-squared:                       0.324
Model:                            OLS   Adj. R-squared:                  0.293
Method:                 Least Squares   F-statistic:                     10.33
Date:                Fri, 06 Dec 2024   Prob (F-statistic):           5.64e-13
Time:                        17:18:25   Log-Likelihood:                -1395.5
No. Observations:                 204   AIC:                             2811.
Df Residuals:                     194   BIC:                             2844.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    216.4879    647.602      0.334      0.739   -1060.756    1493.732
battery_type              32.5098     44.573      0.729      0.467     -55.399     120.419
inches                   110.9896     60.913      1.822      0.070      -9.147     231.126
battery                   -0.0699      0.028     -2.509      0.013      -0.125      -0.015
ram(GB)                  -10.2491     11.729     -0.874      0.383     -33.382      12.883
weight(g)                  2.6868      1.025      2.622      0.009       0.666       4.708
storage(GB)                0.5752      0.455      1.265      0.207      -0.321       1.472
Resolution_product       -74.0547     48.460     -1.528      0.128    -169.631      21.522
days_from_release_date     0.0053      0.042      0.126      0.900      -0.077       0.087
up_to_8K                 182.8745     42.436      4.309      0.000      99.178     266.571
==============================================================================
Omnibus:                      131.154   Durbin-Watson:                   1.938
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1170.554
Skew:                           2.363   Prob(JB):                    6.57e-255
Kurtosis:                      13.741   Cond. No.                     1.85e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.85e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 1.a

Using the t-test computed by `OLS`, we can reject the Null hypothesis of being equal to 0 with the 95% confidence level for:
* battery
* weight(g)
* up_to_8K

### 1.b

In [5]:
lm = LinearRegression() 

scores_loo_SE = cross_val_score(lm, X1, Y, scoring='neg_mean_squared_error', cv=LeaveOneOut())
scores_loo_AE = cross_val_score(lm, X1, Y, scoring='neg_mean_absolute_error', cv=LeaveOneOut())
RMSE_1 = np.sqrt(np.mean(-1*scores_loo_SE))
MAE_1 = np.mean(-1*scores_loo_AE)
print('RMSE:', RMSE_1)
print('MAE:', MAE_1)

RMSE: 239.86636501188858
MAE: 158.04079472828


## Model 2

In [6]:
X2 = data[['inches','battery','ram(GB)','weight(g)','storage(GB)','Resolution_product','days_from_release_date','up_to_8K']]
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X2)
X_comb = np.concatenate((data['battery_type'].values.reshape(-1, 1), X_poly), axis=1)
X_comb = sm.add_constant(X_comb)

In [7]:
model2 = sm.OLS(Y,X_comb).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             price(USD)   R-squared:                       0.541
Model:                            OLS   Adj. R-squared:                  0.411
Method:                 Least Squares   F-statistic:                     4.142
Date:                Fri, 06 Dec 2024   Prob (F-statistic):           1.94e-11
Time:                        17:18:29   Log-Likelihood:                -1356.0
No. Observations:                 204   AIC:                             2804.
Df Residuals:                     158   BIC:                             2957.
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.274e+04   2.22e+04     -0.575      0.566   -5.65e+04     3.1e+04
x1            87.7618     50.431      1.740      0.084     -11.844     187.368
x2          2024.0974   3170.227      0.638      0.524   -4237.393    8285.588
x3             0.7836      2.403      0.326      0.745      -3.962       5.529
x4          -880.3860    729.204     -1.207      0.229   -2320.631     559.859
x5           -23.3065     60.731     -0.384      0.702    -143.255      96.642
x6            23.5598     31.370      0.751      0.454     -38.398      85.518
x7           957.2862   3262.388      0.293      0.770   -5486.231    7400.803
x8             3.0162      2.136      1.412      0.160      -1.203       7.235
x9         -1813.8480   2220.996     -0.817      0.415   -6200.519    2572.823
x10          183.2385    198.484      0.923      0.357    -208.785     575.262
x11            0.3297      0.233      1.413      0.160      -0.131       0.791
x12            4.2539     75.742      0.056      0.955    -145.344     153.852
x13           -5.6467      4.568     -1.236      0.218     -14.670       3.376
x14           -1.5288      3.962     -0.386      0.700      -9.355       6.297
x15         -321.6812    266.456     -1.207      0.229    -847.956     204.594
x16            0.1159      0.211      0.550      0.583      -0.300       0.532
x17          170.5771    282.507      0.604      0.547    -387.400     728.554
x18         3.419e-05   3.58e-05      0.955      0.341   -3.65e-05       0.000
x19            0.0003      0.028      0.009      0.993      -0.055       0.055
x20           -0.0022      0.003     -0.816      0.416      -0.007       0.003
x21           -0.0010      0.001     -1.045      0.298      -0.003       0.001
x22           -0.2025      0.155     -1.309      0.193      -0.508       0.103
x23            0.0002   7.79e-05      3.063      0.003    8.48e-05       0.000
x24           -0.0198      0.088     -0.225      0.822      -0.194       0.154
x25           -1.4657      7.101     -0.206      0.837     -15.491      12.559
x26            0.5768      1.083      0.533      0.595      -1.562       2.715
x27            0.3466      0.300      1.155      0.250      -0.246       0.939
x28           52.6031     50.714      1.037      0.301     -47.561     152.767
x29            0.0235      0.039      0.603      0.547      -0.053       0.100
x30          -49.5865     39.371     -1.259      0.210    -127.348      28.175
x31           -0.0459      0.061     -0.752      0.453      -0.166       0.075
x32            0.0401      0.058      0.688      0.493      -0.075       0.155
x33            5.7853      4.229      1.368      0.173      -2.568      14.139
x34           -0.0037      0.003     -1.058      0.292      -0.011       0.003
x35           -1.5921      3.469     -0.459      0.647      -8.443       5.259
x3

### 2.a

Dummy variables are binary (0 or 1) and do not have a meaningful quadratic relationship. Squaring a dummy variable does not provide additional information because the square of 0 is 0 and the square of 1 is 1. Therefore, including quadratic terms for dummy variables does not contribute to the model's predictive power and can unnecessarily complicate the model. Eventually, quadratic terms and interactions are meaningful for numerical variables, where they can capture non-linear relationships and interactions between continuous variables.

### 2.b

In [8]:
print(model1.rsquared_adj)
print(model2.rsquared_adj)

0.2925710877954183
0.41053197053778234


Based on the in-sample estimation results, we can compare the predictive performance of Model 2 with Model 1:

In both model we might have multicollinearity issue, so we can't compare the models based on thaht. In model 2 the $R^2_{adj}=0.410$, while in model 1 is $R^2_{adj}=0.293$. Higher values indicate a better fit, also considering that $R^2_{adj}$ takes in consideration also the number of variables used in the regression. Since Model 2 has an higher value, it indicates that the model explains the variability in the data more effectively, considering the number of predictors.


### 2.c

In [9]:
lm = LinearRegression() 

scores_loo_SE = cross_val_score(lm, X_comb, Y, scoring='neg_mean_squared_error', cv=LeaveOneOut())
scores_loo_AE = cross_val_score(lm, X_comb, Y, scoring='neg_mean_absolute_error', cv=LeaveOneOut())
RMSE_2 = np.sqrt(np.mean(-1*scores_loo_SE))
MAE_2 = np.mean(-1*scores_loo_AE)
print('RMSE:', RMSE_2)
print('MAE:', MAE_2)

RMSE: 314.83784343476736
MAE: 193.1227118583801


Model 2 have higer RMSE and MAE values when evaluated using LOO-CV, even though it has an higher $R^2_{adj}$. This indicates that Model 2 predicts the data worse than the previous model.

## Model 3

In [10]:
logY = np.log(data['price(USD)'])
model3 = sm.OLS(logY,X1).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             price(USD)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.316
Method:                 Least Squares   F-statistic:                     11.44
Date:                Fri, 06 Dec 2024   Prob (F-statistic):           2.57e-14
Time:                        17:18:30   Log-Likelihood:                -162.03
No. Observations:                 204   AIC:                             344.1
Df Residuals:                     194   BIC:                             377.2
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      6.0092      1.532      3.922      0.000       2.987       9.031
battery_type              -0.0567      0.105     -0.538      0.591      -0.265       0.151
inches                    -0.0193      0.144     -0.134      0.894      -0.304       0.265
battery                   -0.0002   6.59e-05     -2.295      0.023      -0.000   -2.13e-05
ram(GB)                    0.0143      0.028      0.515      0.607      -0.040       0.069
weight(g)                  0.0056      0.002      2.307      0.022       0.001       0.010
storage(GB)                0.0010      0.001      0.944      0.346      -0.001       0.003
Resolution_product        -0.1152      0.115     -1.005      0.316      -0.341       0.111
days_from_release_date   4.15e-05   9.85e-05      0.421      0.674      -0.000       0.000
up_to_8K                   0.4975      0.100      4.954      0.000       0.299       0.696
==============================================================================
Omnibus:                       15.245   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.989
Skew:                           0.436   Prob(JB):                     6.18e-06
Kurtosis:                       4.435   Cond. No.                     1.85e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.85e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 3.a

In [11]:
print(model1.rsquared < model3.rsquared)
print(model1.rsquared_adj < model3.rsquared_adj)

True
True


Based on the comparison of both $R^2$ and $R_{adj}^2$, model 3 is better than model 1. 

### 3.b

In [12]:
lm = LinearRegression() 

scores_loo_SE = cross_val_score(lm, X1, logY, scoring='neg_mean_squared_error', cv=LeaveOneOut())
scores_loo_AE = cross_val_score(lm, X1, logY, scoring='neg_mean_absolute_error', cv=LeaveOneOut())
RMSE_3 = np.sqrt(np.mean(-1*scores_loo_SE))
MAE_3 = np.mean(-1*scores_loo_AE)
print('RMSE:', RMSE_3)
print('MAE:', MAE_3)

RMSE: 0.5635857770463664
MAE: 0.4336832669862257


In [13]:
print(100*(RMSE_3-RMSE_1)/RMSE_1)
print(100*(MAE_3-MAE_1)/MAE_1)

-99.76504176522688
-99.72558777135241


In model 3 we have reduced the error about $100\%$. The model 3 is far better than model 1.

## Model 4

Removing the variables for which we have statistical significance to accept the Null hypothesis in model 3 allows us the create a more robust model. It should lead to a smaller RMSE.

In [14]:
X4 = data[['battery','weight(g)','up_to_8K']]
X4 = sm.add_constant(X4)

lm = LinearRegression()

scores_loo_SE = cross_val_score(lm, X4, logY, scoring='neg_mean_squared_error', cv=LeaveOneOut())
scores_loo_AE = cross_val_score(lm, X4, logY, scoring='neg_mean_absolute_error', cv=LeaveOneOut())
print('RMSE:', np.sqrt(np.mean(-1*scores_loo_SE)))
print('MAE:', np.mean(-1*scores_loo_AE))
print('The new model is better than model 3?',RMSE_3 > np.sqrt(np.mean(-1*scores_loo_SE)))

RMSE: 0.5487058524331132
MAE: 0.42477610273631156
The new model is better than model 3? True


Our model is better than the model 3. The RMSE is lower and the MAE is lower as well.

In [15]:
model4 = sm.OLS(logY,X4).fit()
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             price(USD)   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     34.07
Date:                Fri, 06 Dec 2024   Prob (F-statistic):           7.85e-18
Time:                        17:18:35   Log-Likelihood:                -163.34
No. Observations:                 204   AIC:                             334.7
Df Residuals:                     200   BIC:                             348.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.2915      0.317     13.528      0.000       3.666       4.917
battery       -0.0002   4.84e-05     -3.335      0.001      -0.000   -6.59e-05
weight(g)      0.0063      0.002      2.904      0.004       0.002       0.011
up_to_8K       0.5193      0.059      8.785      0.000       0.403       0.636
==============================================================================
Omnibus:                       14.663   Durbin-Watson:                   1.912
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               20.519
Skew:                           0.467   Prob(JB):                     3.50e-05
Kurtosis:                       4.242   Cond. No.                     3.72e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.72e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
model3.rsquared_adj < model4.rsquared_adj

True

Interestengly, the model is better also in terms of $R^2_{adj}$ (of course, it can't be better in terms of $R^2$, since we are using a third of the variables).

In [17]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data['Feature'] = X4.columns
vif_data['VIF'] = [variance_inflation_factor(X4.values, i) for i in range(X4.shape[1])]
vif_data = vif_data.iloc[1:]
print(vif_data)

     Feature       VIF
1    battery  1.616864
2  weight(g)  1.701755
3   up_to_8K  1.084406


These VIF values indicate that there is no significant multicollinearity among the features in Model 4, as all VIF values are below the commonly used threshold of 5.

Our new model could be better because we have removed some noise from the other variables that are indeed ininfluent ($\beta=0$).